In [ ]:
#Importing Libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.optimizers import Adamax

In [ ]:
np.random.seed(0)

In [ ]:
#encoder
encoder = OneHotEncoder()
encoder.fit([[0], [1]])

In [ ]:
#yes directory
data=[]
result=[]
for file in os.listdir("/kaggle/input/brain-tumor-dataset/brain_tumor_dataset/yes"):
    if file.endswith('.jpg'):
        img = Image.open(os.path.join("/kaggle/input/brain-tumor-dataset/brain_tumor_dataset/yes", file))
        img = img.resize((128, 128))
        img_array = np.array(img)
        if img_array.shape == (128, 128, 3):
            data.append(img_array)
            result.append(encoder.transform([[0]]).toarray())

In [ ]:
#no directory
for file in os.listdir("/kaggle/input/brain-tumor-dataset/brain_tumor_dataset/no"):
    if file.endswith('.jpg'):
        img = Image.open(os.path.join("/kaggle/input/brain-tumor-dataset/brain_tumor_dataset/no", file))
        img = img.resize((128, 128))
        img_array = np.array(img)
        if img_array.shape == (128, 128, 3):
            data.append(img_array)
            result.append(encoder.transform([[1]]).toarray())

In [ ]:
#converting list to numpy array
data = np.array(data)
result = np.array(result).reshape(-1, 2)

In [ ]:
#train test split
x_train, x_test, y_train, y_test = train_test_split(data, result, test_size=0.2, random_state=0)

In [ ]:
#model configuration
model = Sequential()
model.add(Conv2D(32, kernel_size=(2, 2), input_shape=(128, 128, 3), padding='same', activation='relu'))
model.add(Conv2D(32, kernel_size=(2, 2), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, kernel_size=(2, 2), padding='same', activation='relu'))
model.add(Conv2D(64, kernel_size=(2, 2), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [ ]:
#model compilation
model.compile(loss="categorical_crossentropy", optimizer=Adamax(), metrics=['accuracy'])

In [ ]:
#evaluation
print(model.summary())

In [ ]:
#training
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)

In [ ]:
#tested data
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

In [ ]:
#pridiction
def predict_tumor(img_path):
    img = Image.open(img_path)
    img = img.resize((128, 128))
    img_array = np.array(img).reshape(1, 128, 128, 3)
    prediction = model.predict(img_array)
    class_idx = np.argmax(prediction)
    confidence = prediction[0, class_idx]
    
    plt.imshow(img)
    plt.title(f'{confidence * 100:.2f}% Confidence - {class_idx}')
    plt.show()

In [ ]:
predict_tumor("/kaggle/input/brain-tumor-dataset/brain_tumor_dataset/yes/Y100.JPG")